In [1]:
import argparse
import pathlib
import sys

import pandas as pd
from cytotable import convert, presets

sys.path.append("../../../utils")
import uuid

import duckdb
from parsl.config import Config
from parsl.executors import HighThroughputExecutor

try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False

In [2]:
if not in_notebook:
    argparser = argparse.ArgumentParser()
    argparser.add_argument(
        "--patient",
        type=str,
        required=True,
        help="Patient ID to process, e.g. 'P01'",
    )
    argparser.add_argument(
        "--well_fov",
        type=str,
        required=True,
        help="Well and field of view to process, e.g. 'A01_1'",
    )
    args = argparser.parse_args()
    patient = args.patient
    well_fov = args.well_fov
else:
    patient = "NF0014"
    well_fov = "C4-2"

In [3]:
input_sqlite_file = pathlib.Path(
    f"../../data/{patient}/converted_profiles/{well_fov}/{well_fov}.duckdb"
).resolve(strict=True)
destination_sc_parquet_file = pathlib.Path(
    f"../../data/{patient}/image_based_profiles/{well_fov}/sc_profiles_{well_fov}.parquet"
).resolve()
destination_organoid_parquet_file = pathlib.Path(
    f"../../data/{patient}/image_based_profiles/{well_fov}/organoid_profiles_{well_fov}.parquet"
).resolve()
destination_sc_parquet_file.parent.mkdir(parents=True, exist_ok=True)
dest_datatype = "parquet"

In [4]:
# show the tables
con = duckdb.connect(input_sqlite_file)
tables = con.execute("SHOW TABLES").fetchdf()
tables["name"].to_list()

['Cell', 'Cytoplasm', 'Nuclei', 'Organoid']

In [5]:
nuclei_table = con.sql("SELECT * FROM Nuclei").df()
cells_table = con.sql("SELECT * FROM Cell").df()
cytoplasm_table = con.sql("SELECT * FROM Cytoplasm").df()
organoid_table = con.sql("SELECT * FROM Organoid").df()
con.close()

In [6]:
nuclei_id_set = set(nuclei_table["object_id"].to_list())
cells_id_set = set(cells_table["object_id"].to_list())
cytoplasm_id_set = set(cytoplasm_table["object_id"].to_list())
# find the intersection of the three sets
intersection_set = nuclei_id_set.intersection(cells_id_set, cytoplasm_id_set)
# keep only the rows in the three tables that are in the intersection set
nuclei_table = nuclei_table[nuclei_table["object_id"].isin(intersection_set)]
cells_table = cells_table[cells_table["object_id"].isin(intersection_set)]
cytoplasm_table = cytoplasm_table[cytoplasm_table["object_id"].isin(intersection_set)]

In [7]:
con = duckdb.connect()
con.register("df1", nuclei_table)
con.register("df2", cells_table)
con.register("df3", cytoplasm_table)
# Merge them with SQL
merged_df = con.execute("""
    SELECT *
    FROM df1
    LEFT JOIN df2 USING (object_id)
    LEFT JOIN df3 USING (object_id)
""").df()
con.close()

In [8]:
# save the organoid data as parquet
print(f"Final organoid data shape: {merged_df.shape}")
organoid_table.to_parquet(destination_organoid_parquet_file, index=False)
organoid_table.head()

Final organoid data shape: (30, 2106)


object_id image_set  Area.Size.Shape_Organoid_AGP_VOLUME  \
0         32      C4-2                           20908636.0   

   Area.Size.Shape_Organoid_AGP_CENTER.X  \
0                             669.720104   

   Area.Size.Shape_Organoid_AGP_CENTER.Y  \
0                              557.78382   

   Area.Size.Shape_Organoid_AGP_CENTER.Z  \
0                              14.473177   

   Area.Size.Shape_Organoid_AGP_BBOX.VOLUME  \
0                                33828762.0   

   Area.Size.Shape_Organoid_AGP_MIN.X  Area.Size.Shape_Organoid_AGP_MAX.X  \
0                                 167                                1173   

   Area.Size.Shape_Organoid_AGP_MIN.Y  ...  \
0                                  39  ...   

   Texture_Organoid_Mito_Difference.Entropy_256.1  \
0                                         1.33351   

   Texture_Organoid_Mito_Difference.Variance_256.1  \
0                                         0.002369   

   Texture_Organoid_Mito_Entropy_256.1  \
0                              2.76259   

   Texture_Organoid_Mito_Information.Measure.of.Correlation.1_256.1  \
0                                          -0.507738                  

   Texture_Organoid_Mito_Information.Measure.of.Correlation.2_256.1  \
0                                           0.920417                  

   Texture_Organoid_Mito_Inverse.Difference.Moment_256.1  \
0                                           0.831104       

   Texture_Organoid_Mito_Sum.Average_256.1  \
0                                 8.297708   

   Texture_Organoid_Mito_Sum.Entropy_256.1  \
0                                 2.122714   

   Texture_Organoid_Mito_Sum.Variance_256.1  \
0                                198.636846   

   Texture_Organoid_Mito_Variance_256.1  
0                              50.32471  

[1 rows x 702 columns]

In [9]:
print(f"Final merged dataframe shape: {merged_df.shape}")
# save the sc data as parquet
merged_df.to_parquet(destination_sc_parquet_file, index=False)
merged_df.head()

Final merged dataframe shape: (30, 2106)


object_id image_set  Area.Size.Shape_Nuclei_AGP_VOLUME  \
0         15      C4-2                            99661.0   
1         26      C4-2                           156362.0   
2         37      C4-2                            84453.0   
3         43      C4-2                           131041.0   
4         51      C4-2                            69045.0   

   Area.Size.Shape_Nuclei_AGP_CENTER.X  Area.Size.Shape_Nuclei_AGP_CENTER.Y  \
0                           473.778268                           746.900453   
1                           715.339418                           224.483033   
2                           503.486353                           253.488615   
3                           693.588457                           424.839974   
4                           399.909088                           694.579159   

   Area.Size.Shape_Nuclei_AGP_CENTER.Z  \
0                            10.934618   
1                             7.888688   
2                             4.147455   
3                            19.420853   
4                             5.122876   

   Area.Size.Shape_Nuclei_AGP_BBOX.VOLUME  Area.Size.Shape_Nuclei_AGP_MIN.X  \
0                                142417.0                               421   
1                                256768.0                               652   
2                                130980.0                               453   
3                                334530.0                               639   
4                                105984.0                               355   

   Area.Size.Shape_Nuclei_AGP_MAX.X  Area.Size.Shape_Nuclei_AGP_MIN.Y  ...  \
0                               528                               687  ...   
1                               780                               165  ...   
2                               564                               195  ...   
3                               757                               369  ...   
4                               447                               649  ...   

   Texture_Cytoplasm_Mito_Difference.Entropy_256.1  \
0                                         0.051024   
1                                         0.061713   
2                                         0.030409   
3                                         0.051189   
4                                         0.043201   

   Texture_Cytoplasm_Mito_Difference.Variance_256.1  \
0                                          0.003861   
1                                          0.003853   
2                                          0.003874   
3                                          0.003860   
4                                          0.003866   

   Texture_Cytoplasm_Mito_Entropy_256.1  \
0                              0.070853   
1                              0.091123   
2                              0.042813   
3                              0.071099   
4                              0.061412   

   Texture_Cytoplasm_Mito_Information.Measure.of.Correlation.1_256.1  \
0                                          -0.599960                   
1                                          -0.599050                   
2                                          -0.607538                   
3                                          -0.587976                   
4                                          -0.619343                   

   Texture_Cytoplasm_Mito_Information.Measure.of.Correlation.2_256.1  \
0                                           0.242368                   
1                                           0.273598                   
2                                           0.191419                   
3                                           0.239543                   
4                                           0.231458                   

   Texture_Cytoplasm_Mito_Inverse.Difference.Moment_256.1  \
0                                           0.996241        
1                                           0.995213        
